# Setup

In [16]:
# define llm 
llm_config = {"model": "gpt-4-turbo"}

# Task 

In [17]:
task = "Write a blogpost about teh stock price performance of Nvidia in the past month. Today's date is 2024-08-27"

# Group Chat 
- Group char will include these agents:
    1. User_proxy or admin: To allow the user to comment on the report and ask the writer to refine it 
    2. Planner: To determine relevant information needed to complete the task
    3. Engineer: To write code using the defined plan by the planner 
    4. Executor: To execute code written by the engineer
    5. Writer: To write the report 

In [18]:
import autogen 

In [4]:
user_proxy = autogen.ConversableAgent(
    name="user_proxy",
    system_message="Give the task, and send instruction to writer to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [5]:
planner = autogen.ConversableAgent(
    name="planner",
    system_message="Given a task, please determine"
    "What information is needed to complete the task."
    "Please note that the information will all be retrieved using python code."
    "Please only suggest information that can be retrieved using python code."
    "After each step is done by others, check the progress and instruct the remaining steps."
    "If a step fails, try to workaround.",
    description="Planner. Given a task, determine what information is needed to complete the task."
    "After each step is done by others, check the progress and instruct the remaining steps.",
    llm_config=llm_config,
)

In [6]:
engineer = autogen.AssistantAgent(
    name="engineer",
    llm_config=llm_config,
    description="An Engineer that writes code based on the plan provided by the planner"
)

# in this we can use "LocalCommandLineCodeExecutor" like in notebook 5 

In [7]:
executor = autogen.ConversableAgent(
    name="executor",
    system_message="Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding_report_generation",
        "use_docker": False,
    }
)

In [8]:
writer = autogen.ConversableAgent(
    name="writer",
    llm_config=llm_config,
    system_message="Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block."
    "You take feedback from the main admin or user_proxy and refine your blog.",
    description="Writer. Write blogs based on the code execution results and take feedback from the admin or user_proxy to refine the blog."
)

# Define the group chat 

In [10]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10
)

In [11]:
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

# Start the group chat 

In [12]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task
)

user_proxy (to chat_manager):

Write a blogpost about teh stock price performance of Nvidia in the past month. Today's date is 2024-08-27

--------------------------------------------------------------------------------

Next speaker: planner


>>>>>>>> USING AUTO REPLY...
planner (to chat_manager):

To write a blog post about the stock price performance of Nvidia over the past month, using Python, the essential tasks are as follows:

1. **Retrieve Historical Stock Data**: Gather Nvidia's stock price data for the past month.

2. **Analyze the Data**: Calculate statistics such as the opening price, closing price, high, low, and any significant changes or anomalies in the stock price.

3. **Visualize the Data**: Generate graphs to visually interpret the stock's performance over the selected period.

4. **Interpret the Data**: Analyze reasons behind significant changes in the stock price (e.g., company announcements, market trends, economic factors).

5. **Draft the Blog Post**: Using the

# Add a speaker selection policy 

In [20]:
user_proxy = autogen.ConversableAgent(
    name="user_proxy",
    system_message="Give the task, and send instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="planner",
    system_message="Given a task, please determine what information is needed to complete the task."
    "Please note that the information will all be retrieved using python code. Please only suggest information that can be retrieved using python code."
    "After each step is done by others, check the progress and instruct the remaining step. If a step fails, try to workaround",
    description="Given a task, determine what information is needed to comple the task."
    "After each step is done by others, check the progress and instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.ConversableAgent(
    name="engineer",
    llm_config=llm_config,
    description="Write code based on the plan privided by the planner."
)

writer = autogen.ConversableAgent(
    name="writer",
    llm_config=llm_config,
    system_message="Writer. Please write blogs in markdown format (with relevant titles) and put the content in pseudo ```md``` code block."
    "You take feedback from the admin or user_proxy and refine your blog.",
    description="Given a task, determine what information is needed to complete the task."
    "After each step is done by others, check the progress and instruct the remaining steps",
)

executor = autogen.ConversableAgent(
    name="executor",
    description="Execute the code written by the engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_message": 3,
        "work_dir": "coding_report_generation",
        "use_docker": False,
    },
)

In [21]:
grouchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer]
    },
    speaker_transitions_type="allowed",
)

In [23]:
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    manager=task,
)

user_proxy (to chat_manager):

Write a blogpost about the bitcoin price performance in the pas month. Today's date is 2024-08-27

--------------------------------------------------------------------------------

Next speaker: planner


>>>>>>>> USING AUTO REPLY...
planner (to chat_manager):

To write a blog post about Bitcoin's price performance over the past month using Python, you'll need to gather and analyze relevant data. Here’s the step-by-step process and the information needed to achieve this:

### Step 1: Collect Bitcoin Price Data
- **Information Needed:**
  1. Historical daily Bitcoin prices for the past month (from 2024-07-27 to 2024-08-27).
  2. Any relevant financial news or events that might have influenced Bitcoin prices during this period.

### Step 2: Analyze the Price Data
- **Information Needed:**
  1. Significant price changes (e.g., highest and lowest prices within the month).
  2. Overall trend (whether the price was generally increasing, decreasing, or stable).
